In [154]:
#!pip install webdriver_manager
#!pip install webdriver_manager --upgrade

#!pip install selenium --upgrade
!#webdriver_manager chrome --clear-cache



In [155]:
import os
import re
import sys
import time
import numpy as np
import pandas as pd
import gurobipy as gp
from gurobipy import GRB
from datetime import datetime
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager


In [156]:
chrome_driver_path = '/Users/coltonlapp/chromedriver'

In [157]:
parameters = {
    'name': 'ElectricityArbitrage',
    'generator_name': 'ADK HUDSON___FALLS',
    'start_date': '20231124',
    'num_periods': 24,
    'num_markets': 1,
    'num_batteries': 8,
    'battery_capacity': 100,
    'charge_loss': 0.95,
    'max_charge': 50,
    'max_discharge': 100
}

In [158]:
def extract_date(file_path):
    match = re.match(r'^(\d+)', file_path.split('/')[4])

    return match.group(1)

In [159]:
def get_prices(start_date, generator_name):
    if start_date == None:
        start_date = datetime.today()
        start_date = start_date.strftime("%Y%m%d")

    dates = [start_date]

    driver_path = ChromeDriverManager().install()

    home_dir = os.path.expanduser("~")

    download_directory = os.path.join(home_dir, 'Downloads_CSV')
    chrome_options = webdriver.ChromeOptions()
    prefs = {'download.default_directory': download_directory}
    chrome_options.add_experimental_option('prefs', prefs)

    try:
        #driver = webdriver.Chrome(executable_path=driver_path, options=chrome_options)
        #driver = webdriver.Chrome(executable_path=chrome_driver_path, options=chrome_options)
        driver=webdriver.Chrome()
    except:
        try:
            driver = webdriver.Chrome(ChromeDriverManager().install())
        except:
            sys.exit('Error: could not find a Chrome webdriver.')

    driver.get('http://mis.nyiso.com/public/P-24Blist.htm')
    all_links = driver.find_elements("xpath", "//a[@href]")

    if not os.path.exists(download_directory):
        os.makedirs(download_directory)

    for date in dates:
        for link in all_links:
            href = link.get_attribute('href')

            if ('csv' in href) and (date in href):
                link.click()
                time.sleep(2)
                break

    csv_files = [file for file in os.listdir(download_directory) if file.endswith('.csv')]
    prices_by_date = {}

    for file_name in csv_files:
        file_path = os.path.join(download_directory, file_name)
        date = extract_date(file_path)
        df = pd.read_csv(file_path)
        prices = df[df['Name'] == generator_name][['LBMP ($/MWHr)']].values.tolist()

        prices_by_date[date] = prices

    driver.quit()

    return prices_by_date

In [160]:
def create_model(parameters): # num_markets is going to be 1 for the time being
    name = parameters['name']
    generator_name = parameters['generator_name']
    start_date = parameters['start_date']
    num_periods = parameters['num_periods']
    num_markets = parameters['num_markets']
    num_batteries = parameters['num_batteries']
    battery_capacity = parameters['battery_capacity']
    charge_loss = parameters['charge_loss']
    max_charge = parameters['max_charge']
    max_discharge = parameters['max_discharge']

    model = gp.Model(name)

    periods = range(num_periods)
    markets = range(num_markets)

    buy = model.addVars(num_periods, num_markets, vtype=GRB.CONTINUOUS, name='Buy')
    sell = model.addVars(num_periods, num_markets, vtype=GRB.CONTINUOUS, name='Sell')


    prices_by_date = get_prices(start_date, generator_name)
    print(prices_by_date)
    
    # placeholder
    start_date = '20231123'
    #start_date = datetime.today()
    #start_date = start_date.strftime("%Y%m%d")

    print(prices_by_date)
    print(start_date)

    prices = prices_by_date[start_date]

    model.setObjective(
        gp.quicksum(prices[p][i] * sell[p, i] - prices[p][i] * buy[p, i] for p in periods for i in markets),
        GRB.MAXIMIZE
    )

    for p in range(num_periods):
        current_level = np.sum(charge_loss * buy[p_, i] - sell[p_, i] for p_ in range(p) for i in markets)

        # model.addConstr(current_level <= 0, f'EnoughToSellConstraint_period_{p+1}')

        for i in markets:
            model.addConstr(current_level <= battery_capacity * num_batteries, f'CapacityConstraint_period_{p+1}')
            model.addConstr(sell[p, i] <= current_level, f'SupplyConstraint_period_{p+1}')
            model.addConstr(buy[p, i] * charge_loss <= max_charge, f'ChargeConstraint_period_{p+1}')
            model.addConstr(sell[p, i] <= max_discharge, f'DischargeConstraint_period_{p+1}')

    return [model, buy, sell]

In [161]:
def print_model(model, buy, sell, parameters):
    num_periods = parameters['num_periods']
    num_markets = parameters['num_markets']

    if model.status == GRB.OPTIMAL:
        print("\nOptimal Solution:")

        for p in range(num_periods):
            print(f"\nPeriod {p + 1}:")

            for i in range(num_markets):
                print(f"Buy from Market {i + 1}: {buy[p, i].x}")
                print(f"Sell to Market {i + 1}: {sell[p, i].x}")

        print(f"\nTotal Profit: {model.objVal}")
    else:
        print("No solution found")

In [162]:
def run(parameters):
    [model, buy, sell] = create_model(parameters)

    model.optimize()

    print_model(model, buy, sell, parameters)

    return [model, buy, sell]

In [163]:
model, buy, sell = run(parameters)

{}
{}
20231123


KeyError: '20231123'